- Introdução
- Ementa
- O que esperar/o que não esperar desse tutorial
- O que é ETL
- Fonte de dados
- Extração
- Exploração inicial
- Tratamento
- Carregamento

# Introdução 

In [1]:
# TODO

# O que esperar/o que não esperar desse tutorial 

## O que esperar 

1. Conceitos básicos de engenharia de dados
2. Como implementar rotinas de extração de dados usando da lib Requests
3. Como manipular dados tabulares com o Pandas
4. Como analisar uma fonte de dados 
5. Como organizar a experimentação com small data em python

## O que não esperar 

1. Implementação de pipelines de streaming de dados
2. Solução completa pronta para produção
3. Como construir pipelines utilizando ferramentas de workflow(ex.: Airflow)
4. Como trabalhar com Big data em python

# O que é ETL

In [3]:
# TODO

# A fonte de dados 

Vamos trabalhar com a fonte de dados abertos da CVM contendo a cotação diária dos fundos de investimentos negociados no mercado brasileiro. Essa fonte é atualizada diariamente com os dados de fechamento do dia anterior e as cotações são agrupadas por arquivos correspondentes a cada mês do ano.

Esse é um cenário bem comum em fontes de dados abertas, uma série de arquivos no formato CSV agrupando informações de acordo com a data, então a solução que vamos implementar durante o tutorial é reaproveitável para outras fontes de dados.

Primeiro é importante analisar a fonte de dados, entender como ela está estruturada, quais campos compõem o dataset e como nós podemos automatizar a coleta dos dados.

A fonte de dados contendo a cotação diária dos fundos pode ser acessada através do portal de dados abertos pelo link:

http://www.dados.gov.br/dataset/fi-doc-inf_diario

## Como automatizar o download 

Podemos observar que no site existe um link para todos os datasets dos últimos 12 meses. Primeiro precisamos analisar se existe um padrão nas urls de download dos arquivos, para isso vamos copiar alguns endereços e compará-los.

Copiei três links e vamos compará-los a seguir:

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv

Como podemos observar existe um padrão claro nos links:

`dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_YYYYMM.csv`

# ETL

In [21]:
import pandas as pd
import requests
from tqdm import tqdm

## Extração 

O processo de extração consiste nesse caso em fazer o download de todos os arquivos da janela de tempo q nos interessa. As etapas pra nós atingirmos esse objetivo são:

1. Automatizar a geração do nome dos arquivos: já que a única coisa que varia nos links é o nome dos arquivos precisamos automatizar a geração desses nomes de acordo com a janela de tempo do nosso interesse;

2. Requisitar o arquivo: precisamos enviar uma requisição para o portal de dados abertos do arquivo que queremos fazer o download;

3. Salvar arquivo: o portal de dados abertos vai nos enviar o arquivo requisitado e precisamos salvá-lo no nosso computador.

### Gerando o nome dos arquivos 

In [13]:
def generate_dates(start, end, freq='M', format='%Y%m'):
    return [date.strftime(format) for date in pd.date_range(start=start, end=end, freq=freq)]

In [19]:
date_range = generate_dates('2020-01', '2020-03')

### Requisitano e salvando os arquivos 

In [24]:
def download_files(dates, save_dir='data'):
    saved_files = []
    
    file_base_url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{date}.csv'
    
    for date in tqdm(dates):
        url = file_base_url.format(date=date)
        
        response = requests.get(url)
        
        if response.status_code != 200:
            print('Erro {status} ao requisitar o arquivo: {file}'.format(status=response.status_code, file=url))
            continue
        
        file_save_name = '{dir}/inf_diario_fi_{date}.csv'.format(dir=save_dir, date=date)
        
        with open(file_save_name, 'wb') as f:
            f.write(response.content)
        
        saved_files.append(file_save_name)
    
    return saved_files

In [26]:
files = download_files(date_range)

100%|██████████| 2/2 [00:35<00:00, 17.76s/it]


## Análise

// TODO descrição
Aqui vou abrir um parênteses para uma análise rápido do nosso dateframe, é claro que só a parte de exploração dos dados vale um tutorial completo, então não vou explorar muito esses aspecto aqui, mas de qualquer forma é importa ter noção de alguns pontos básicos quando se trabalha com uma fonte de dados:

- Qual o tipo de cada coluna
- Quantos valores nulos que existem
- Como estão formatados

In [28]:
def concat_csvs(file_list, sep=','):
    df_list = [pd.read_csv(file, sep=sep) for file in file_list]
    return pd.concat(df_list)

In [29]:
raw_df = concat_csvs(files, sep=';')

In [30]:
# Formato do dataframe (linhas, colunas)
raw_df.shape

(675754, 8)

In [32]:
# Quais são nossas colunas, seus tipos e se existem valores nulos
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675754 entries, 0 to 305859
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   CNPJ_FUNDO     675754 non-null  object 
 1   DT_COMPTC      675754 non-null  object 
 2   VL_TOTAL       675754 non-null  float64
 3   VL_QUOTA       675754 non-null  float64
 4   VL_PATRIM_LIQ  675754 non-null  float64
 5   CAPTC_DIA      675754 non-null  float64
 6   RESG_DIA       675754 non-null  float64
 7   NR_COTST       675754 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 46.4+ MB


In [33]:
raw_df.head()

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2020-01-02,1132491.66,27.225023,1123583.00,0.0,0.0,1
1,00.017.024/0001-53,2020-01-03,1132685.12,27.224496,1123561.25,0.0,0.0,1
2,00.017.024/0001-53,2020-01-06,1132881.43,27.225564,1123605.31,0.0,0.0,1
3,00.017.024/0001-53,2020-01-07,1133076.85,27.226701,1123652.24,0.0,0.0,1
4,00.017.024/0001-53,2020-01-08,1132948.59,27.227816,1123698.26,0.0,0.0,1


## Tratamento 